In [1]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 140-96*x+12*x*x
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 100
    child_num = 50

    space = OrderedDict((
        ('x', (-2., 150.)),
        ('y', (0., 0.)),
        ('z', (0., 0.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 56.95119149319727, 0.0, 0.0: 33593.94416659097
Eval 92.28244423786956, 0.0, 0.0: 93473.47952735057
Eval 140.32111319456493, 0.0, 0.0: 222949.35083126463
Eval 73.41547286550863, 0.0, 0.0: 57770.094477705985
Eval 112.99967701695688, 0.0, 0.0: 142519.155077611
Eval 20.31505178596568, 0.0, 0.0: 3142.1709773449024
Eval 4.354864221678592, 0.0, 0.0: -50.48885661006935
Eval 143.9751602709626, 0.0, 0.0: 235064.54591457997
Eval 138.44520984863595, 0.0, 0.0: 216854.17341492508
Eval 141.92579269483696, 0.0, 0.0: 228230.29148598967
Eval 91.13359826015432, 0.0, 0.0: 91055.16734914361
Eval 57.73552926965683, 0.0, 0.0: 34598.08527068175
Eval 102.4188009465903, 0.0, 0.0: 116183.12455717476
Eval 103.44025824684064, 0.0, 0.0: 118608.3795223803
Eval 132.0983352437347, 0.0, 0.0: 196858.20190659497
Eval 93.70866511249403, 0.0, 0.0: 96519.73515518726
Eval 66.49665224017042, 0.0, 0.0: 46817.97849474558
Eval 31.36346929411699, 0.0, 0.0: 8933.113421721
Eval 91.37574338187487, 0.0, 0.0: 91562.24637842299
Ev